In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
import os
import torch
import torchvision
import torchvision.transforms as transforms

In [4]:
dataset_path_raw = '/content/drive/MyDrive/PRML Project/covid 19/Combinedmodel_Dataset/raw_cxr/'
dataset_path_masked = '/content/drive/MyDrive/PRML Project/covid 19/Combinedmodel_Dataset/masked_cxr/'
dataset_path_invmasked = '/content/drive/MyDrive/PRML Project/covid 19/Combinedmodel_Dataset/invmasked_cxr/'

In [5]:
from torchvision.transforms.transforms import RandomHorizontalFlip
dataset_transforms2 = transforms.Compose([
                                          transforms.Resize((224,224)), 
                                          transforms.RandomHorizontalFlip(),
                                          transforms.RandomRotation(10),
                                          transforms.ToTensor(),
                                        ])

In [6]:
dataset_raw = torchvision.datasets.ImageFolder(root = dataset_path_raw, transform = dataset_transforms2)
dataset_masked = torchvision.datasets.ImageFolder(root = dataset_path_masked, transform = dataset_transforms2)
dataset_invmasked = torchvision.datasets.ImageFolder(root = dataset_path_invmasked, transform = dataset_transforms2)

In [7]:
batch_size = 4

In [8]:
len(dataset_raw), len(dataset_masked), len(dataset_invmasked)

(5000, 5000, 5000)

In [9]:
train_size = int(0.8 * len(dataset_raw))
validation_size = int(0.1 * len(dataset_raw))
test_size = len(dataset_raw) - train_size - validation_size
train_dataset_raw, validation_dataset_raw, test_dataset_raw = torch.utils.data.random_split(dataset_raw, [train_size,validation_size,test_size])

In [10]:
train_loader_raw = torch.utils.data.DataLoader(dataset = train_dataset_raw, batch_size = batch_size)
valid_loader_raw = torch.utils.data.DataLoader(dataset = validation_dataset_raw, batch_size = batch_size)
test_loader_raw = torch.utils.data.DataLoader(dataset = test_dataset_raw, batch_size = batch_size)

In [11]:
train_size = int(0.8 * len(dataset_masked))
validation_size = int(0.1 * len(dataset_masked))
test_size = len(dataset_masked) - train_size - validation_size
train_dataset_masked, validation_dataset_masked, test_dataset_masked = torch.utils.data.random_split(dataset_masked, [train_size,validation_size,test_size])

In [12]:
train_loader_masked = torch.utils.data.DataLoader(dataset = train_dataset_masked, batch_size = batch_size)
valid_loader_masked = torch.utils.data.DataLoader(dataset = validation_dataset_masked, batch_size = batch_size)
test_loader_masked = torch.utils.data.DataLoader(dataset = test_dataset_masked, batch_size = batch_size)

In [13]:
train_size = int(0.7 * len(dataset_invmasked))
validation_size = int(0.2 * len(dataset_invmasked))
test_size = len(dataset_invmasked) - train_size - validation_size
train_dataset_invmasked, validation_dataset_invmasked, test_dataset_invmasked = torch.utils.data.random_split(dataset_invmasked, [train_size,validation_size,test_size])

In [14]:
train_loader_invmasked = torch.utils.data.DataLoader(dataset = train_dataset_invmasked, batch_size = batch_size)
valid_loader_invmasked = torch.utils.data.DataLoader(dataset = validation_dataset_invmasked, batch_size = batch_size)
test_loader_invmasked = torch.utils.data.DataLoader(dataset = test_dataset_invmasked, batch_size = batch_size)

In [15]:
# for (i1, (f1,l1)), (i2, (f2,l2)), (i3, (f3,l3)) in zip(enumerate(train_loader_raw), enumerate(train_loader_masked), enumerate(train_loader_invmasked)):
#   print("----------------")
#   print(i1)
#   print(f1)
#   print("----------------")
#   print(i2)
#   print(f2)
#   print("----------------")
#   print(i3)
#   print(f3)
#   break

In [31]:
for batch_idx, (features, targets) in enumerate(train_loader_raw):
  print(batch_idx)
  print(features.size())
  print(targets.size())
  print(targets)
  break

0
torch.Size([4, 3, 224, 224])
torch.Size([4])
tensor([1, 0, 0, 0])


In [ ]:
for batch_idx, (features, targets) in enumerate(train_loader_masked):
  print(batch_idx)
  print(features.size())
  print(targets.size())
  print(targets)
  break

In [ ]:
k=0
lis = []
for i, (f,l) in enumerate(train_loader_raw):
  lis.append((i, (f,l)))

In [32]:
for (batch_idx1, (features1, targets1)), (batch_idx2, (features2,targets2)), (batch_idx3, (features3,targets3)) in zip(enumerate(train_loader_raw), enumerate(train_loader_masked), enumerate(train_loader_invmasked)):
  print(batch_idx1)
  print(features1.size())
  print(targets1.size())
  print(targets1)
  break



0
torch.Size([4, 3, 224, 224])
torch.Size([4])
tensor([0, 0, 0, 0])


In [17]:
# imports from installed libraries
import os
import numpy as np
import random
import torch
from distutils.version import LooseVersion as Version
from itertools import product

def set_all_seeds(seed):
    os.environ["PL_GLOBAL_SEED"] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)


def set_deterministic(use_tensorcores=False):
    if torch.cuda.is_available():
        torch.backends.cudnn.benchmark = False
        torch.backends.cudnn.deterministic = True

    if torch.__version__ <= Version("1.7"):
        torch.set_deterministic(True)
    else:
        torch.use_deterministic_algorithms(True)
        
        # The following are set to True by default and allow cards
        # like the Ampere and newer to utilize tensorcores for
        # convolutions and matrix multiplications, which can result
        # in a significant speed-up. However, results may differ compared
        # to card how don't use mixed precision via tensor cores.
        torch.backends.cuda.matmul.allow_tf32 = use_tensorcores
        torch.backends.cudnn.allow_tf32 = use_tensorcores


def compute_accuracy(model, data_loader_raw, data_loader_masked, data_loader_invmasked, device):

    with torch.no_grad():

        correct_pred, num_examples = 0, 0

        for (i1, (features1, targets1)), (i2, (features2,targets2)), (i3, (features3,targets3)) in zip(enumerate(train_loader_raw), enumerate(train_loader_masked), enumerate(train_loader_invmasked)):

            i = i1
            features1 = features1.to(device)
            features2 = features2.to(device)
            features3 = features3.to(device)
            targets = targets1.float().to(device)

            logits = model(features1, features2, features3)
            _, predicted_labels = torch.max(logits, 1)

            num_examples += targets.size(0)
            correct_pred += (predicted_labels == targets).sum()
    return correct_pred.float()/num_examples * 100


def compute_confusion_matrix(model, data_loader_raw, data_loader_masked, data_loader_invmasked, device):

    all_targets, all_predictions = [], []
    with torch.no_grad():

        for (i1, (features1, targets1)), (i2, (features2,targets2)), (i3, (features3,targets3)) in zip(enumerate(train_loader_raw), enumerate(train_loader_masked), enumerate(train_loader_invmasked)):
            i = i1
            features1 = features1.to(device)
            features2 = features2.to(device)
            features3 = features3.to(device)
            targets = targets1
            logits = model(features1, features2, features3)
            _, predicted_labels = torch.max(logits, 1)
            all_targets.extend(targets.to('cpu'))
            all_predictions.extend(predicted_labels.to('cpu'))

    all_predictions = all_predictions
    all_predictions = np.array(all_predictions)
    all_targets = np.array(all_targets)
        
    class_labels = np.unique(np.concatenate((all_targets, all_predictions)))
    if class_labels.shape[0] == 1:
        if class_labels[0] != 0:
            class_labels = np.array([0, class_labels[0]])
        else:
            class_labels = np.array([class_labels[0], 1])
    n_labels = class_labels.shape[0]
    lst = []
    z = list(zip(all_targets, all_predictions))
    for combi in product(class_labels, repeat=2):
        lst.append(z.count(combi))
    mat = np.asarray(lst)[:, None].reshape(n_labels, n_labels)
    return mat

In [18]:
import time
import torch

def train_model(model, num_epochs, train_loader_raw,
                valid_loader_raw, test_loader_raw, train_loader_masked,
                valid_loader_masked, test_loader_masked, train_loader_invmasked,
                valid_loader_invmasked, test_loader_invmasked, optimizer,
                device, logging_interval=50,
                scheduler=None,
                scheduler_on='valid_acc'):

    start_time = time.time()
    minibatch_loss_list, train_acc_list, valid_acc_list = [], [], []
    
    best_acc = 0
    for epoch in range(num_epochs):

        model.train()
        for (batch_idx1, (features1, targets1)), (batch_idx2, (features2,targets2)), (batch_idx3, (features3,targets3)) in zip(enumerate(train_loader_raw), enumerate(train_loader_masked), enumerate(train_loader_invmasked)):
            
            # batch_idx1, batch_idx2, and batch_idx3 are all same.
            batch_idx = batch_idx1
            features1 = features1.to(device)
            features2 = features2.to(device)
            features3 = features3.to(device)
            # targets1, targets2, and target3 are all same.
            targets = targets1.to(device)

            # ## FORWARD AND BACK PROP
            logits = model(features1, features2, features3)
            loss = torch.nn.functional.cross_entropy(logits, targets)
            optimizer.zero_grad()

            loss.backward()

            # ## UPDATE MODEL PARAMETERS
            optimizer.step()

            # ## LOGGING
            minibatch_loss_list.append(loss.item())
            if not batch_idx % logging_interval:
                print(f'Epoch: {epoch+1:03d}/{num_epochs:03d} '
                      f'| Batch {batch_idx:04d}/{len(train_loader_raw):04d} '
                      f'| Loss: {loss:.4f}')

        model.eval()
        with torch.no_grad():  # save memory during inference
            train_acc = compute_accuracy(model, train_loader_raw, train_loader_masked, train_loader_invmasked, device=device)
            valid_acc = compute_accuracy(model, valid_loader_raw, valid_loader_masked, valid_loader_invmasked, device=device)
            print(f'Epoch: {epoch+1:03d}/{num_epochs:03d} '
                  f'| Train: {train_acc :.2f}% '
                  f'| Validation: {valid_acc :.2f}%')
            train_acc_list.append(train_acc.item())
            valid_acc_list.append(valid_acc.item())

        if(valid_acc.item() > best_acc):
          best_acc = valid_acc.item()
          torch.save(model.state_dict(), f'/content/drive/MyDrive/PRML Project/covid 19/M4_weights/m4_02_05_{valid_acc.item()}.pt')

        elapsed = (time.time() - start_time)/60
        print(f'Time elapsed: {elapsed:.2f} min')
        
        if scheduler is not None:

            if scheduler_on == 'valid_acc':
                scheduler.step(valid_acc_list[-1])
            elif scheduler_on == 'minibatch_loss':
                scheduler.step(minibatch_loss_list[-1])
            else:
                raise ValueError(f'Invalid `scheduler_on` choice.')
        

    elapsed = (time.time() - start_time)/60
    print(f'Total Training Time: {elapsed:.2f} min')

    test_acc = compute_accuracy(model, test_loader_raw, test_loader_masked, test_loader_invmasked, device=device)
    print(f'Test accuracy {test_acc :.2f}%')

    return minibatch_loss_list, train_acc_list, valid_acc_list

In [19]:
# imports from installed libraries
import os
import matplotlib.pyplot as plt
import numpy as np
import torch


def plot_training_loss(minibatch_loss_list, num_epochs, iter_per_epoch,
                       results_dir=None, averaging_iterations=100):

    plt.figure()
    ax1 = plt.subplot(1, 1, 1)
    ax1.plot(range(len(minibatch_loss_list)),
             (minibatch_loss_list), label='Minibatch Loss')

    if len(minibatch_loss_list) > 1000:
        ax1.set_ylim([
            0, np.max(minibatch_loss_list[1000:])*1.5
            ])
    ax1.set_xlabel('Iterations')
    ax1.set_ylabel('Loss')

    ax1.plot(np.convolve(minibatch_loss_list,
                         np.ones(averaging_iterations,)/averaging_iterations,
                         mode='valid'),
             label='Running Average')
    ax1.legend()

    ###################
    # Set scond x-axis
    ax2 = ax1.twiny()
    newlabel = list(range(num_epochs+1))

    newpos = [e*iter_per_epoch for e in newlabel]

    ax2.set_xticks(newpos[::10])
    ax2.set_xticklabels(newlabel[::10])

    ax2.xaxis.set_ticks_position('bottom')
    ax2.xaxis.set_label_position('bottom')
    ax2.spines['bottom'].set_position(('outward', 45))
    ax2.set_xlabel('Epochs')
    ax2.set_xlim(ax1.get_xlim())
    ###################

    plt.tight_layout()

    if results_dir is not None:
        image_path = os.path.join(results_dir, 'plot_training_loss.pdf')
        plt.savefig(image_path)


def plot_accuracy(train_acc_list, valid_acc_list, results_dir):

    num_epochs = len(train_acc_list)

    plt.plot(np.arange(1, num_epochs+1),
             train_acc_list, label='Training')
    plt.plot(np.arange(1, num_epochs+1),
             valid_acc_list, label='Validation')

    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()

    plt.tight_layout()

    if results_dir is not None:
        image_path = os.path.join(
            results_dir, 'plot_acc_training_validation.pdf')
        plt.savefig(image_path)


def show_examples(model, data_loader, unnormalizer=None, class_dict=None):
    
        
    for batch_idx, (features, targets) in enumerate(data_loader):

        with torch.no_grad():
            features = features
            targets = targets
            logits = model(features)
            predictions = torch.argmax(logits, dim=1)
        break

    fig, axes = plt.subplots(nrows=1, ncols=5,
                             sharex=True, sharey=True)
    
    if unnormalizer is not None:
        for idx in range(features.shape[0]):
            features[idx] = unnormalizer(features[idx])
    nhwc_img = np.transpose(features, axes=(0, 2, 3, 1))
    
    if nhwc_img.shape[-1] == 1:
        nhw_img = np.squeeze(nhwc_img.numpy(), axis=3)

        for idx, ax in enumerate(axes.ravel()):
            ax.imshow(nhw_img[idx], cmap='binary')
            if class_dict is not None:
                ax.title.set_text(f'P: {class_dict[predictions[idx].item()]}'
                                  f'\nT: {class_dict[targets[idx].item()]}')
            else:
                ax.title.set_text(f'P: {predictions[idx]} | T: {targets[idx]}')
            ax.axison = False

    else:

        for idx, ax in enumerate(axes.ravel()):
            ax.imshow(nhwc_img[idx])
            if class_dict is not None:
                ax.title.set_text(f'P: {class_dict[predictions[idx].item()]}'
                                  f'\nT: {class_dict[targets[idx].item()]}')
            else:
                ax.title.set_text(f'P: {predictions[idx]} | T: {targets[idx]}')
            ax.axison = False
    plt.tight_layout()
    plt.show()


def plot_confusion_matrix(conf_mat,
                          hide_spines=False,
                          hide_ticks=False,
                          figsize=None,
                          cmap=None,
                          colorbar=False,
                          show_absolute=True,
                          show_normed=False,
                          class_names=None):

    if not (show_absolute or show_normed):
        raise AssertionError('Both show_absolute and show_normed are False')
    if class_names is not None and len(class_names) != len(conf_mat):
        raise AssertionError('len(class_names) should be equal to number of'
                             'classes in the dataset')

    total_samples = conf_mat.sum(axis=1)[:, np.newaxis]
    normed_conf_mat = conf_mat.astype('float') / total_samples

    fig, ax = plt.subplots(figsize=figsize)
    ax.grid(False)
    if cmap is None:
        cmap = plt.cm.Blues

    if figsize is None:
        figsize = (len(conf_mat)*1.25, len(conf_mat)*1.25)

    if show_normed:
        matshow = ax.matshow(normed_conf_mat, cmap=cmap)
    else:
        matshow = ax.matshow(conf_mat, cmap=cmap)

    if colorbar:
        fig.colorbar(matshow)

    for i in range(conf_mat.shape[0]):
        for j in range(conf_mat.shape[1]):
            cell_text = ""
            if show_absolute:
                cell_text += format(conf_mat[i, j], 'd')
                if show_normed:
                    cell_text += "\n" + '('
                    cell_text += format(normed_conf_mat[i, j], '.2f') + ')'
            else:
                cell_text += format(normed_conf_mat[i, j], '.2f')
            ax.text(x=j,
                    y=i,
                    s=cell_text,
                    va='center',
                    ha='center',
                    color="white" if normed_conf_mat[i, j] > 0.5 else "black")
    
    if class_names is not None:
        tick_marks = np.arange(len(class_names))
        plt.xticks(tick_marks, class_names, rotation=90)
        plt.yticks(tick_marks, class_names)
        
    if hide_spines:
        ax.spines['right'].set_visible(False)
        ax.spines['top'].set_visible(False)
        ax.spines['left'].set_visible(False)
        ax.spines['bottom'].set_visible(False)
    ax.yaxis.set_ticks_position('left')
    ax.xaxis.set_ticks_position('bottom')
    if hide_ticks:
        ax.axes.get_yaxis().set_ticks([])
        ax.axes.get_xaxis().set_ticks([])

    plt.xlabel('predicted label')
    plt.ylabel('true label')
    return fig, ax

In [20]:
import torch
from torch.utils.data import sampler
from torchvision import datasets
from torch.utils.data import DataLoader
from torch.utils.data import SubsetRandomSampler
from torchvision import transforms


class UnNormalize(object):
    def __init__(self, mean, std):
        self.mean = mean
        self.std = std

    def __call__(self, tensor):
        """
        Parameters:
        ------------
        tensor (Tensor): Tensor image of size (C, H, W) to be normalized.
        
        Returns:
        ------------
        Tensor: Normalized image.
        """
        for t, m, s in zip(tensor, self.mean, self.std):
            t.mul_(s).add_(m)
        return tensor

In [21]:
import torch
from torch import nn
import torchvision
import numpy as np
import matplotlib.pyplot as plt

In [22]:
RANDOM_SEED = 123
BATCH_SIZE = 8
NUM_EPOCHS = 40
DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [23]:
set_all_seeds(RANDOM_SEED)
#set_deterministic()

In [24]:
DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [25]:
class scratch_vgg19(nn.Module):
	#  Determine what layers and their order in CNN object 
    def __init__(self):
        super(scratch_vgg19, self).__init__()
        self.conv_layer1 = nn.Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        self.relu1 = nn.ReLU()
        self.max_pool1 = nn.MaxPool2d(kernel_size = 2, stride = 2)

        self.conv_layer2 = nn.Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        self.relu2 = nn.ReLU()
        self.max_pool2 = nn.MaxPool2d(kernel_size = 2, stride = 2)

        self.conv_layer3 = nn.Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        self.relu3 = nn.ReLU()
        self.max_pool3 = nn.MaxPool2d(kernel_size = 2, stride = 2)

        self.conv_layer4 = nn.Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        self.relu4 = nn.ReLU()
        self.max_pool4 = nn.MaxPool2d(kernel_size = 2, stride = 2)

        self.flat = nn.Flatten()
        self.fc1 = nn.Linear(100352, 4096)
        self.fc2 = nn.Linear(4096, 4096)
        self.fc3 = nn.Linear(4096, 2)
    
    def forward(self, x):

        out = self.conv_layer1(x)
        out = self.relu1(out)
        out = self.max_pool1(out)

        out = self.conv_layer2(out)
        out = self.relu2(out)
        out = self.max_pool2(out)
                
        out = self.conv_layer3(out)
        out = self.relu3(out)
        out = self.max_pool3(out)
        
        out = self.conv_layer4(out)
        out = self.relu4(out)
        out = self.max_pool4(out)
        
        out = self.flat(out)
        out = self.fc1(out)
        out = self.fc2(out)
        out = self.fc3(out)

        return out

In [26]:
model_raw = scratch_vgg19()
model_raw.load_state_dict(torch.load('/content/drive/MyDrive/PRML Project/covid 19/Weights_full_training/vgg19_full_30_04_93.34032440185547.pt')) # saurabh
model_raw.to(DEVICE)

model_mask = scratch_vgg19()
model_mask.load_state_dict(torch.load('/content/drive/MyDrive/PRML Project/covid 19/Unet_M2_weights/vgg19_full_unetmask_01_05_87.2756118774414.pt')) # mitarth
model_mask.to(DEVICE)

model_inv_mask = scratch_vgg19()
model_inv_mask.load_state_dict(torch.load('/content/drive/MyDrive/PRML Project/covid 19/Unet_M3_weights/vgg19_full_unetinvmask_01_05_91.28828430175781.pt')) # mukul
model_inv_mask.to(DEVICE)

scratch_vgg19(
  (conv_layer1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu1): ReLU()
  (max_pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv_layer2): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu2): ReLU()
  (max_pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv_layer3): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu3): ReLU()
  (max_pool3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv_layer4): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu4): ReLU()
  (max_pool4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (flat): Flatten(start_dim=1, end_dim=-1)
  (fc1): Linear(in_features=100352, out_features=4096, bias=True)
  (fc2): Linear(in_features=4096, out_features=4096, bias=True)
  (fc3): Linear(in_features=4096, out_featur

In [27]:
# code to do the finetuning on the inference weights
# torch.grad() types
# reference
# https://pytorch.org/tutorials/beginner/finetuning_torchvision_models_tutorial.html

In [28]:
import torch.nn as nn
import torch

class CombinedModelM4(nn.Module):
  def __init__(self, model_raw, model_mask, model_inv_mask):
    super().__init__()
    # Note that the last layer for classification should be removed in both the models

    # self.rawvgg = model_raw[:-4]
    self.rawvgg  = nn.Sequential(*list(model_raw.children())[:-3])
    self.maskvgg = nn.Sequential(*list(model_mask.children())[:-3])
    self.invmaskvgg = nn.Sequential(*list(model_inv_mask.children())[:-3])

    self.classifier = nn.Sequential(
                                                  nn.Linear(in_features = 301056, out_features = 4096, bias = True),
                                                  nn.ReLU(inplace = True),
                                                  nn.Linear(in_features = 4096, out_features = 4096, bias = True),
                                                  nn.ReLU(inplace=True),
                                                  nn.Linear(in_features = 4096, out_features = 2, bias = True),
                                              )
    
  def forward(self, cxr_raw,cxr_mask, cxr_inv):

    rawvgg = self.rawvgg(cxr_raw).view(cxr_raw.shape[0], -1)
    maskvgg = self.maskvgg(cxr_mask).view(cxr_mask.shape[0], -1)
    inv_maskvgg = self.invmaskvgg(cxr_inv).view(cxr_inv.shape[0], -1)

    combined_ft = torch.cat([rawvgg, maskvgg, inv_maskvgg], axis = -1)

    out = self.classifier(combined_ft)

    return out

In [29]:
model = CombinedModelM4(model_raw, model_mask, model_inv_mask)
model

CombinedModelM4(
  (rawvgg): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU()
    (8): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (9): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (10): ReLU()
    (11): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (12): Flatten(start_dim=1, end_dim=-1)
  )
  (maskvgg): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3)

In [30]:
# all False and then unfreeze classifier layers
for param in model.parameters():
  param.requires_grad = False
  
model.classifier.requires_grad = True

In [31]:
model = model.to(DEVICE)

optimizer = torch.optim.SGD(model.parameters(), momentum=0.9, lr=0.01)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer,
                                                       factor=0.1,
                                                       mode='max',
                                                       verbose=True)

minibatch_loss_list, train_acc_list, valid_acc_list = train_model(
    model=model,
    num_epochs=NUM_EPOCHS,
    train_loader_raw=train_loader_raw,
    valid_loader_raw=valid_loader_raw,
    test_loader_raw=test_loader_raw,
    train_loader_masked=train_loader_masked,
    valid_loader_masked=valid_loader_masked,
    test_loader_masked=test_loader_masked,
    train_loader_invmasked=train_loader_invmasked,
    valid_loader_invmasked=valid_loader_invmasked,
    test_loader_invmasked=test_loader_invmasked,
    optimizer=optimizer,
    device=DEVICE,
    scheduler=scheduler,
    scheduler_on='valid_acc',
    logging_interval=100)

plot_training_loss(minibatch_loss_list=minibatch_loss_list,
                   num_epochs=NUM_EPOCHS,
                   iter_per_epoch=len(train_loader_raw),
                   results_dir=None,
                   averaging_iterations=200)
plt.show()

plot_accuracy(train_acc_list=train_acc_list,
              valid_acc_list=valid_acc_list,
              results_dir=None)
plt.ylim([60, 100])
plt.show()

RuntimeError: ignored

In [ ]:
torch.save(model.state_dict(), '/content/drive/MyDrive/PRML Project/covid 19/M4_weights/m4_02_05_final.pth')

In [ ]:
model.cpu()
# unnormalizer = UnNormalize(mean, std)
class_dict = {0: 'Covid',
              1: 'Non-Covid'}
show_examples(model=model, data_loader=test_loader_raw, unnormalizer=None, class_dict=class_dict)

In [ ]:
mat = compute_confusion_matrix(model=model, data_loader=test_loader_raw, device=torch.device('cpu'))
plot_confusion_matrix(mat, class_names=class_dict.values())
plt.show()

Dataset Preparation

In [ ]:
import os

for covid

In [ ]:
craw_path = '/content/drive/MyDrive/PRML Project/covid 19/Extensive COVID-19 X-Ray and CT Chest Images Dataset.zip (Unzipped Files)/COVID-19 Dataset.zip (Unzipped Files)/COVID-19 Dataset/X-ray/COVID/'
cmasked_path = '/content/drive/MyDrive/PRML Project/covid 19/masks/Covid/'
cinv_masked_path = '/content/drive/MyDrive/PRML Project/covid 19/inverse_masks/Covid/'

In [ ]:
craw = np.array(os.listdir(craw_path))
cmask = np.array(os.listdir(cmasked_path))
cinv_mask = np.array(os.listdir(cinv_masked_path))

In [ ]:
len(craw), len(cmask), len(cinv_mask)

For Non-Covid

In [ ]:
nraw_path = '/content/drive/MyDrive/PRML Project/covid 19/Extensive COVID-19 X-Ray and CT Chest Images Dataset.zip (Unzipped Files)/COVID-19 Dataset.zip (Unzipped Files)/COVID-19 Dataset/X-ray/Non-COVID/'
nmasked_path = '/content/drive/MyDrive/PRML Project/covid 19/masks/Non-Covid/'
ninv_masked_path = '/content/drive/MyDrive/PRML Project/covid 19/inverse_masks/Non-Covid/'

In [ ]:
nraw = np.array(os.listdir(nraw_path))
nmask = np.array(os.listdir(nmasked_path))
ninv_mask = np.array(os.listdir(ninv_masked_path))

In [ ]:
len(nraw), len(nmask), len(ninv_mask)

Thus now best is to drop this extra 5500 - 5428 = 72 images from Non-Covid directory of cxr raw images

In [ ]:
os.remove('/content/drive/MyDrive/PRML Project/testing/streptococcus-pneumoniae-pneumonia-temporal-evolution-1-day3.jpg')

In [ ]:
nrawl = list(nraw)
nmaskl = list(nmask)
ninvmaskl = list(ninv_mask)

In [ ]:
nraw_files = [os.path.splitext(i)[0] for i in nrawl]
nmask_files = [os.path.splitext(i)[0] for i in nmaskl]
ninvmask_files = [os.path.splitext(i)[0] for i in ninvmaskl] 

In [ ]:
nraw_ext = [os.path.splitext(i)[1] for i in nrawl]
nmask_ext = [os.path.splitext(i)[1] for i in nmaskl]
ninvmask_ext = [os.path.splitext(i)[1] for i in ninvmaskl] 

In [ ]:
rem_base_path = nraw_path

In [ ]:
rem_base_path + nraw_files[0] + nraw_ext[0]

In [ ]:
k=0
rem_count=0
for i in nraw_files:
  if i not in nmask_files:
    os.remove(rem_base_path + i + nraw_ext[k])
    rem_count+=1
  k+=1

In [ ]:
rem_count

In [ ]:
len(nraw_files)

In [ ]:
len(nmask_files)

In [ ]:
for i in nraw_files:
  print(i)

In [ ]:
for i in nmask_files:
  print(i)

In [ ]:
if 'NORMAL2-IM-0906-0001' in nmask_files:
  print("True")

In [ ]:
k=0
add_count=0
add=[]
for i in nmask_files:
  if i in nraw_files:
    # os.remove(rem_base_path + i + nraw_ext[k])
    add.append(i)
    add_count+=1
  k+=1

In [ ]:
len(add)

In [ ]:
rem_list = []
for i in add:
  if i not in nraw_files:
    rem_list.append(i)

In [ ]:
len(np.unique(nraw_files)), len(np.unique(nmask_files))

In [ ]:
l1 = [os.path.splitext(i)[0] for i in list(os.listdir('/content/drive/MyDrive/PRML Project/testing'))]

In [ ]:
l1

In [ ]:
nuraw = list(np.unique(nraw_files))
numasks = list(np.unique(nmask_files))

In [ ]:
raw_rem_list = []
for i in nraw_files:
  if i not in nuraw:
    raw_rem_list.append(i)

In [ ]:
len(raw_rem_list)

In [ ]:
len(nraw_files), len(nuraw)

In [ ]:
n1, n2 = np.array(nraw_files), np.array(nuraw)

In [ ]:
difference_1 = np.setdiff1d(n1, n2)

In [ ]:
difference_1

preparation of dataset of 2500 noncovid images

In [ ]:
raw_files = list(np.unique(list(os.listdir('/content/drive/MyDrive/PRML Project/covid 19/Extensive COVID-19 X-Ray and CT Chest Images Dataset.zip (Unzipped Files)/COVID-19 Dataset.zip (Unzipped Files)/COVID-19 Dataset/X-ray/Non-COVID/'))))
masked_files = list(np.unique(list(os.listdir('/content/drive/MyDrive/PRML Project/covid 19/masks/Non-Covid/'))))
invmasked_files = list(np.unique(list(os.listdir('/content/drive/MyDrive/PRML Project/covid 19/inverse_masks/Non-Covid/'))))

In [ ]:
len(raw_files), len(masked_files), len(invmasked_files)

In [ ]:
import random
idx = random.sample(range(5428), 2500)

In [ ]:
len(idx)

In [ ]:
k=0
for i in raw_files:
  if (i in masked_files) and (i in invmasked_files):
    k+=1

In [ ]:
k

In [ ]:
files_list = []
k=0
for i in raw_files:
  if (i in masked_files) and (i in invmasked_files):
    files_list.append(i)
    k+=1
  if(k==2500):
    break

In [ ]:
len(files_list)

In [ ]:
import cv2

In [ ]:
for i in files_list:
  raw_img = cv2.imread('/content/drive/MyDrive/PRML Project/covid 19/Extensive COVID-19 X-Ray and CT Chest Images Dataset.zip (Unzipped Files)/COVID-19 Dataset.zip (Unzipped Files)/COVID-19 Dataset/X-ray/Non-COVID/' + i)
  masked_img = cv2.imread('/content/drive/MyDrive/PRML Project/covid 19/masks/Non-Covid/' + i)
  invmasked_img = cv2.imread('/content/drive/MyDrive/PRML Project/covid 19/inverse_masks/Non-Covid/' + i)

  cv2.imwrite('/content/drive/MyDrive/PRML Project/covid 19/Combinedmodel_Dataset/raw_cxr/Non-Covid/' + i, raw_img)    
  cv2.imwrite('/content/drive/MyDrive/PRML Project/covid 19/Combinedmodel_Dataset/masked_cxr/Non-Covid/' + i, masked_img)    
  cv2.imwrite('/content/drive/MyDrive/PRML Project/covid 19/Combinedmodel_Dataset/invmasked_cxr/Non-Covid/' + i, invmasked_img)    

In [ ]:
len(os.listdir('/content/drive/MyDrive/PRML Project/covid 19/Combinedmodel_Dataset/raw_cxr/Non-Covid/'))
len(os.listdir('/content/drive/MyDrive/PRML Project/covid 19/Combinedmodel_Dataset/masked_cxr/Non-Covid/'))
len(os.listdir('/content/drive/MyDrive/PRML Project/covid 19/Combinedmodel_Dataset/invmasked_cxr/Non-Covid/'))

2500 Images for Covid

In [ ]:
import numpy as np
import os
import cv2

In [ ]:
craw_files = list(np.unique(list(os.listdir('/content/drive/MyDrive/PRML Project/covid 19/Extensive COVID-19 X-Ray and CT Chest Images Dataset.zip (Unzipped Files)/COVID-19 Dataset.zip (Unzipped Files)/COVID-19 Dataset/X-ray/COVID/'))))
cmasked_files = list(np.unique(list(os.listdir('/content/drive/MyDrive/PRML Project/covid 19/masks/Covid/'))))
cinvmasked_files = list(np.unique(list(os.listdir('/content/drive/MyDrive/PRML Project/covid 19/inverse_masks/Covid/'))))

In [ ]:
len(craw_files), len(cmasked_files), len(cinvmasked_files)

In [ ]:
k=0
for i in craw_files:
  if (i in cmasked_files) and (i in cinvmasked_files):
    k+=1

In [ ]:
k

In [ ]:
import random
idx = random.sample(range(4044), 2500)

In [ ]:
len(np.unique(idx))

In [ ]:
cfiles_list = []
k=0
for i in craw_files:
  if (i in cmasked_files) and (i in cinvmasked_files):
    cfiles_list.append(i)
    k+=1
  if(k==2500):
    break

In [ ]:
len(cfiles_list)

In [ ]:
for i in cfiles_list:
  craw_img = cv2.imread('/content/drive/MyDrive/PRML Project/covid 19/Extensive COVID-19 X-Ray and CT Chest Images Dataset.zip (Unzipped Files)/COVID-19 Dataset.zip (Unzipped Files)/COVID-19 Dataset/X-ray/COVID/' + i)
  cmasked_img = cv2.imread('/content/drive/MyDrive/PRML Project/covid 19/masks/Covid/' + i)
  cinvmasked_img = cv2.imread('/content/drive/MyDrive/PRML Project/covid 19/inverse_masks/Covid/' + i)

  cv2.imwrite('/content/drive/MyDrive/PRML Project/covid 19/Combinedmodel_Dataset/raw_cxr/Covid/' + i, craw_img)    
  cv2.imwrite('/content/drive/MyDrive/PRML Project/covid 19/Combinedmodel_Dataset/masked_cxr/Covid/' + i, cmasked_img)    
  cv2.imwrite('/content/drive/MyDrive/PRML Project/covid 19/Combinedmodel_Dataset/invmasked_cxr/Covid/' + i, cinvmasked_img)    

In [ ]:
len(os.listdir('/content/drive/MyDrive/PRML Project/covid 19/Combinedmodel_Dataset/raw_cxr/Covid/'))

In [ ]:
len(os.listdir('/content/drive/MyDrive/PRML Project/covid 19/Combinedmodel_Dataset/masked_cxr/Covid/'))

In [ ]:
len(os.listdir('/content/drive/MyDrive/PRML Project/covid 19/Combinedmodel_Dataset/invmasked_cxr/Covid/'))